In [1]:
import numpy as np
import pandas as pd
import scipy.spatial
from pixell import utils, enmap, enplot, curvedsky, reproject, fft
import os
from orphics import sehgal
import healpy as hp
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [15]:
catalog_file = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/radsrc/radio.npy"

'''
The columns ar
ra between 0 and 90
dec between 0 and 90
z
f_1.4
f_30
f_90
f_148
f_219
f_277
f_350
(the fluxes are in mJy)
'''

input_cat = np.load(catalog_file)
input_cat[:,:2] *= utils.degree
input_cat[:,3:] *= 1e-3  # convert to Jy
input_cat[:,[0,1]] = input_cat[:,[1,0]]## note ra and dec column swap here
pos = input_cat[:,:2]
#nfullsource = full_cat.shape[0]
loc = np.where((pos[:,0]>=0.)&(pos[:,0]<=np.pi/2.)&(pos[:,1]>=0.)&(pos[:,1]<=np.pi/2.))
input_cat = input_cat[loc[0],:]
nfullsource = input_cat.shape[0]
print(input_cat.shape[0]/nfullsource)
columns = ["dec", "ra", "z", "f1.4", "f30", "f90", "f148", "f219", "f277", "f350"]
index = range(input_cat.shape[0])
print(columns)

#input_cat[:,6] = np.arange(input_cat.shape[0])

1.0
['dec', 'ra', 'z', 'f1.4', 'f30', 'f90', 'f148', 'f219', 'f277', 'f350']


In [16]:
## healpy ang:  0 < ra < 2pi , 0 < dec < pi
## pixell ang: -pi < ra < pi, -pi/2 < dec < pi/2

def pix2hp(pos):
    ret = np.zeros(pos.shape)
    ret[:,0] = pos[:,0]+np.pi/2.
    ret[:,1] = pos[:,1]+np.pi
    return ret
    
def hp2pix(pos):
    ret = np.zeros(pos.shape)
    ret[:,0] = pos[:,0] -np.pi/2.
    ret[:,1] = pos[:,1] -np.pi
    return ret   

In [22]:
hp.pixelfunc.nside2resol(8192/2, True)

0.8588716029543515

In [17]:
shape, wcs = enmap.fullsky_geometry(res=0.5*utils.arcmin)
ycent = shape[0] // 2
ydelta = int(10*utils.degree/(0.5*utils.arcmin))
ymin, ymax = ycent-ydelta, ycent+ydelta

template = enmap.zeros(shape, wcs)
template = template[ymin:ymax,:].copy()
shape, wcs = template.geometry
print(shape, ycent, ydelta, ymin, ymax)

template = enmap.zeros(shape, wcs)
areamap = template.pixsizemap()


(2400, 43200) 10800 1200 9600 12000


In [13]:
is_flux_limited = True
flux_cut = 20e-3 if is_flux_limited else np.inf

overwrite = False
output_dir = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/July2009/output/from_radio_cat"
def output_path(x):
    return os.path.join(output_dir, x)

os.makedirs(output_dir, exist_ok=True)

rot_angles1 = [0]#, 15, 30, 45, 60, 75]
#rot_angles2 = [0, 20, 40, 60, 80]
#rot_angles1 = [30]
rot_angles2 = [0]#, 20, 40, 60, 80]
#rot_angles2 = [20]
#test_ctr = 0
#test = {}
for rot_angle1 in rot_angles1:
    for rot_angle2 in rot_angles2:
        file_path = output_path("148_%s_enmap_%s_%s_000.fits")
        file_path = file_path %("rad_pts", "%0.3d"%rot_angle1, "%0.3d"%rot_angle2)
        if os.path.exists(file_path) and not overwrite:
            continue
        if rot_angle2 == 0 and rot_angle1 != 0:
            continue
        else:
            pass
        print(file_path)
        
        ctr = 0
        full_sky = template.copy()
        for xidx in [-2,-1,0,1]:
            for yidx in [-1,1]:
                print(ctr,rot_angle1, rot_angle2, xidx,yidx)

                full_cat = input_cat.copy()
                if yidx == -1:#full_sky
                    ## flip at dec = 0
                    full_cat[:,0] *= -1 
                    ## flip at ra = np.pi
                    full_cat[:,1] -= np.pi/4
                    full_cat[:,1] *= -1
                    full_cat[:,1] += np.pi/4
                else:
                    pass
                full_cat[:,1] += (np.pi/2)*xidx ## shift quadrant
                ctr += 1

                hppos = pix2hp(full_cat[:,:2])

                uvec = hp.ang2vec(hppos[:,0],hppos[:,1])
                #mat_rot, _, _ = hp.rotator.get_rotation_matrix((np.pi/4,np.pi/4,0))
                mat_rot, _, _ = hp.rotator.get_rotation_matrix((rot_angle1*utils.degree*-1,rot_angle2*utils.degree,0))
                rot_vec = np.inner(mat_rot, uvec).T

                pixpos = np.zeros(hppos.shape)
                temp = hp.vec2ang(rot_vec)
                pixpos[:,0] = temp[0]
                pixpos[:,1] = temp[1]
                pixpos = hp2pix(pixpos)        
                pix = np.round(enmap.sky2pix(shape, wcs , pixpos.T).T) 

                loc = np.where((pix[:,0]>=0)&(pix[:,0]<shape[0])&(pix[:,1]>=0.)&(pix[:,1]<shape[1]))
                pix = pix[loc[0],:]
                normalized_cat = full_cat[loc[0],:].copy()
                normalized_cat[:,:2] = pix
                normalized_cat = np.delete(normalized_cat, 2, axis=1) ## drop z column
                columns_norm = ["y", "x", "f1.4", "f30", "f90", "f148", "f219", "f277", "f350"]
                temp = {}
                for i in range(normalized_cat.shape[0]):
                    #if i %  (len(normalized_cat)//5) == 0: print(i /  (len(normalized_cat)//5))
                    y, x =  normalized_cat[i,:2].astype(int)
                    key = (y,x)
                    if key in temp:
                        temp[key].append(i)
                    else:
                        temp[key] = [i]


                merged_cat = np.zeros((len(temp),9))
                for i, key in enumerate(temp):
                    subidx = temp[key]
                    if len(subidx) == 1:
                        merged_cat[i,:] = normalized_cat[subidx[0],:]
                    else:
                        merged_cat[i,:2] = key
                        merged_cat[i,2:] = np.sum(normalized_cat[subidx,2:], axis=0)   
                print("fraction of sources being merged :", (merged_cat.shape[0])/input_cat.shape[0])
                del temp

                loc = np.where(merged_cat[:,5]<=flux_cut)
                fluxcut_cat = merged_cat[loc[0],:]
                #fluxcut_cat = merged_cat
                print("num of sources being cut :", (merged_cat.shape[0]-len(loc[0])))
                pix = fluxcut_cat[:,:2].astype(np.int)
                full_sky[pix[:,0], pix[:,1]] = fluxcut_cat[:,5]

        full_sky = full_sky/areamap
        full_sky *= sehgal.jysr2thermo(148)
        #test[test_ctr] = full_sky.copy()
        #test_ctr += 1
        enmap.write_map(file_path, full_sky)
        #print(np.max(full_sky))

In [27]:
(0.5*utils.arcmin)**2/hp.pixelfunc.nside2pixarea(8192)

1.3556373079549675

In [25]:
assert(True)

rot_angles1 = [0]#, 15, 30, 45, 60, 75]
rot_angles2 = [0, 20, 40, 60, 80]
#rot_angles1 = [30]
#rot_angles2 = [20]
newproj_dir = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/July2009/output/from_radio_cat"
oldproj_dir = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/July2009/output/rotated_lmax10000_20mjycuts"

for rot_angle1 in rot_angles1:
    for rot_angle2 in rot_angles2:
        newproj_path = os.path.join(newproj_dir, "148_%s_enmap_%s_%s_000.fits")
        newproj_path = newproj_path %("rad_pts", "%0.3d"%rot_angle1, "%0.3d"%rot_angle2)
        oldproj_path = os.path.join(oldproj_dir, "148_%s_alm_%s_%s_000.fits")
        oldproj_path = oldproj_path %("rad_pts", "%0.3d"%rot_angle1, "%0.3d"%rot_angle2)
        if rot_angle2 == 0 and rot_angle1 != 0:
            continue
        assert(os.path.exists(newproj_path))
        assert(os.path.exists(oldproj_path))
        print(rot_angle1, rot_angle2)
        
        newproj = enmap.read_fits(newproj_path)
        oldproj = template.copy()
        oldproj = curvedsky.alm2map(np.complex128(hp.read_alm(oldproj_path)), oldproj)
        
        
        print("new total flux", np.sum(newproj*areamap))
        print("old total flux", np.sum(oldproj*areamap))
        #print("old max flux", np.max(oldproj*areamap))
        print("max flux ratio", np.max(newproj*areamap)/np.max(oldproj*areamap))
            
        '''
        alm1 = curvedsky.map2alm(newproj, lmax=10000)
        alm2 = curvedsky.map2alm(oldproj, lmax=10000)

        cl1 = hp.alm2cl(alm1)
        cl2 = hp.alm2cl(alm2)
        clx = hp.alm2cl(alm1, alm2)
        
        def cl2dl(cl):
            l = np.arange(len(cl))
            l_fact = l*(l+1)/(2*np.pi)
            return l, l_fact*cl

        l, dl1 = cl2dl(cl1)
        l, dl2 = cl2dl(cl2)
        l, dlx = cl2dl(clx)
        rho = dlx/np.sqrt(dl1*dl2)
        
        fig = plt.figure(figsize=(7,5))
        ax = plt.gca()
        plt.plot(l,dl1, label="projection from cat")
        plt.plot(l,dl2, label="projection from S16")
        plt.plot(l,dlx, label="Cross")
        plt.title("{} degree in RA {} degree in Dec rotation".format(rot_angle1, rot_angle2))
        plt.legend(fontsize=25)
        plt.xlabel(r"$\ell$", fontsize=15)
        ax.tick_params(axis='both', which='major', labelsize=15)
        ax.tick_params(axis='both', which='minor', labelsize=15)
        plt.show()
        
        fig = plt.figure(figsize=(7,5))
        ax = plt.gca()
        plt.plot(l,rho, label="corr coeff")
        plt.legend(fontsize=25)
        plt.xlabel(r"$\ell$", fontsize=15)
        ax.tick_params(axis='both', which='major', labelsize=15)
        ax.tick_params(axis='both', which='minor', labelsize=15)
        plt.title("{} degree in RA {} degree in Dec rotation".format(rot_angle1, rot_angle2))
        plt.ylim(0.8, 1.2)
        plt.show()
        '''
            

0 0
new total flux 0.6938907735375406
old total flux 0.6939443455011821
old max flux 8.447895953232965e-06
old max flux 6.000717519718793
0 20
new total flux 0.6771722317654496
old total flux 0.6772358741666713
old max flux 8.415037372591165e-06
old max flux 6.039865622873265
0 40
new total flux 0.6770571403179853
old total flux 0.6772583875851899
old max flux 8.458362415801143e-06
old max flux 5.995125267564194
0 60
new total flux 0.678038383445074
old total flux 0.6781204464027901
old max flux 8.409383211916249e-06
old max flux 6.04392660687483
0 80
new total flux 0.6825317322746315
old total flux 0.6827482844909806
old max flux 8.382917565546693e-06
old max flux 6.06251268193923


In [85]:
## KD trea based apporach. Will work better for CEA projection

assert(False)


KD = scipy.spatial.cKDTree(full_cat[:,:2]) 

shape, wcs = enmap.fullsky_geometry(res=0.5*utils.arcmin)
template = enmap.zeros(shape, wcs)
template_cea, _= reproject.centered_map(template, res=0.5*utils.arcmin, proj="cea")
template = enmap.submap(template, [[0,0],[np.pi/2, np.pi/2]])
template_cea = enmap.submap(template_cea, [[0,0],[np.pi/2, np.pi/2]])
posmap_cea = enmap.posmap(template_cea.shape, template_cea.wcs) # posmap goes from -90 to 90 in dec and -180 to 180 in ra
pixsizemap_cea =  enmap.pixsizemap(template_cea.shape, template_cea.wcs)

points = np.c_[posmap[1].ravel(),posmap[0].ravel() ]
areas = np.c_[pixsizemap.ravel()]
query_ball = KD.query_ball_point(points, 0.2821*utils.arcmin)
filtered_query_ball = [(points[i],areas[i],query) for i, query in enumerate(query_ball) if len(query) != 0]

overwrite = True
projected_catalog_file = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/radsrc/radio_projected_0.5arcmin.npy"
if not os.path.exists(projected_catalog_file) or overwrite:
    projected_cat = np.zeros((len(query_ball), 9))
    area = (0.5*utils.arcmin)**2
    for i, source_idx in enumerate(query_ball):
        point = points[i]
        projected_cat[i][0] = point[1] #"dec"
        projected_cat[i][1] = point[0] #"ra"
        if len(source_idx) != 0:
            projected_cat[i][2:9] = np.sum(full_cat[source_idx, 3:], axis=0)/areas[i]        
        else:
            pass
    np.save(projected_catalog_file, projected_cat)
else:
    projected_cat = np.load(projected_catalog_file)
    
### convert the flux from mJy to Jy/stradian
overwrite = True
filtered_catalog_file = "/home/dwhan89/scratch/data/sehgal_et_al_sims/cosmo_sim_maps/radsrc/radio_projected_0.5arcmin_filtered.npy"
if not os.path.exists(filtered_catalog_file) or overwrite:
    filtered_cat = np.zeros((len(filtered_query_ball), 9))
    area = (0.5*utils.arcmin)**2
    for i, query in enumerate(filtered_query_ball):
        tpoints, tarea, source_idx = query
        filtered_cat[i][0] = tpoints[1] #"dec"
        filtered_cat[i][1] = tpoints[0] #"ra"
        filtered_cat[i][2:9] = np.sum(full_cat[source_idx, 3:], axis=0)/tarea  
    np.save(filtered_catalog_file, filtered_cat)
else:
    filtered_cat = np.load(filtered_catalog_file)

for i, freq_idx in enumerate(["f1.4", "f30", "f90", "f148", "f219", "f277", "f350"]):
    freq_idx = float(freq_idx[1:])
    jysr2thermo =  sehgal.jysr2thermo(freq_idx)
    filtered_cat[:,i+2] = filtered_cat[:,i+2]*jysr2thermo
    
## apply 20mJy flux cut at 148GHz
#area = (0.5*utils.arcmin)**2
flux_cut = 20*1e-3/area #20mJy over stradian
flux_cut *= sehgal.jysr2thermo(148)
filtered_cat_fullflux = filtered_cat.copy()
loc = np.where(filtered_cat[:,5] <= flux_cut)
filtered_cat = filtered_cat[loc[0],:]
 
fullsky = enmap.zeros(shape, wcs)
for i in [-2,-1,0,1]:
    for j in [-1,1]:
        temp = filtered_cat.copy()
        if j == -1:
            ## flip at dec = 0
            temp[:,0] *= -1 
            ## flip at ra = np.pi
            temp[:,1] -= np.pi/4
            temp[:,1] *= -1
            temp[:,1] += np.pi/4
        else:
            pass
        
        temp[:,1] += (np.pi/2)*i ## shift quadrant
        pix = enmap.sky2pix(shape,wcs, temp[:,:2].T).T.astype(int)
        fullsky[pix[:,0], pix[:,1]] = temp[:,5]
        

fullsky_fullflux = enmap.zeros(shape, wcs)
for i in [-2,-1,0,1]:
    for j in [-1,1]:
        temp = filtered_cat_fullflux.copy()
        if j == -1:
            ## flip at dec = 0
            temp[:,0] *= -1 
            ## flip at ra = np.pi
            temp[:,1] -= np.pi/4
            temp[:,1] *= -1
            temp[:,1] += np.pi/4
        else:
            pass
        
        temp[:,1] += (np.pi/2)*i ## shift quadrant
        pix = enmap.sky2pix(shape,wcs, temp[:,:2].T).T.astype(int)
        fullsky_fullflux[pix[:,0], pix[:,1]] = temp[:,5]
            

SyntaxError: invalid syntax (<ipython-input-85-e2a4b58b1cf4>, line 3)